<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled264.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Clean the environment and install core libraries
!pip install -q gemmi pyscf

# 2. Re-install CuPy (Force clean install of the core dependency)
!pip uninstall -y cupy-cuda11x cupy-cuda12x # Ensure cleanup
!pip install -q cupy-cuda12x # Install the modern, clean version

# 3. Disable verbose GPU warnings that slow down the run
import os
os.environ['CUPY_PACKAGE_NAME'] = 'cupy'


Found existing installation: cupy-cuda11x 13.6.0
Uninstalling cupy-cuda11x-13.6.0:
  Successfully uninstalled cupy-cuda11x-13.6.0
Found existing installation: cupy-cuda12x 13.6.0
Uninstalling cupy-cuda12x-13.6.0:
  Successfully uninstalled cupy-cuda12x-13.6.0


In [ ]:
# -*- coding: utf-8 -*-
# ==============================================================================
# Fe4S4 CURVATURE TRAP PIPELINE - STABLE VERSION (WITH GPU ACCELERATION)
# ==============================================================================

import os
import sys
import urllib.request
import numpy as np
import gemmi

# PySCF Imports: Relying on base modules + to_gpu() method
from pyscf import gto, dft
from pyscf.dft import numint

# ------------------------------------------
# 1. CONFIGURATION (Unchanged)
# ------------------------------------------
BASIS_SET = 'def2-svp'
FUNCTIONAL = 'pbe'
GRID_POINTS = 32
BOX_MARGIN = 3.5
DISPLACEMENT = 0.20
DIIS_MAX_CYCLE = 50
CONV_TOL = 1e-5
L_EFF = 3.0
BINS_LIST = list(range(1, 126, 1))

# Setup environment
project_root = "/content/gqr_fe4s4_ispG"
os.makedirs(project_root, exist_ok=True)
os.chdir(project_root)
print("Working directory:", os.getcwd())

# ------------------------------------------
# 2. CLASS: STRUCTURE EXTRACTOR (Unchanged)
# ------------------------------------------
class FeS_Analyzer:
    def __init__(self, cif_path):
        self.cif_path = cif_path
        try:
            self.structure = gemmi.read_structure(cif_path)
        except Exception as e:
            print(f"Error reading CIF: {e}")
            sys.exit(1)
        self.model = self.structure[0]
        self.atoms = []

    def extract_cluster(self, residue_name="SF4"):
        print(f" [Loader] Reading {self.cif_path}...")
        cluster_atoms = []

        for chain in self.model:
            for res in chain:
                if residue_name in res.name:
                    for atom in res:
                        pos = atom.pos
                        cluster_atoms.append({
                            'elem': atom.element.name,
                            'pos': np.array([pos.x, pos.y, pos.z])
                        })

        if not cluster_atoms:
            raise ValueError(f"Cluster extraction failed: '{residue_name}' not found.")

        positions = np.array([a['pos'] for a in cluster_atoms])
        centroid = np.mean(positions, axis=0)
        self.atoms = []
        for atom in cluster_atoms:
            self.atoms.append((atom['elem'], atom['pos'] - centroid))
        print(f" Extracted {len(self.atoms)} atoms and centered geometry.")

        symbols = np.array([a[0] for a in self.atoms])
        coords_centered = np.array([a[1] for a in self.atoms])
        return symbols, coords_centered

    def apply_mode_b_distortion(self, symbols, coords_centered):
        new_positions = coords_centered.copy()
        fe_indices = [i for i, s in enumerate(symbols) if s == 'Fe']
        s_indices = [i for i, s in enumerate(symbols) if s == 'S']

        pairs = []
        for fe_idx in fe_indices:
            fe_pos = coords_centered[fe_idx]
            min_dist = np.inf
            nearest_s_idx = -1
            for s_idx in s_indices:
                dist = np.linalg.norm(fe_pos - coords_centered[s_idx])
                if dist < min_dist:
                    min_dist = dist
                    nearest_s_idx = s_idx
            pairs.append((fe_idx, nearest_s_idx))

        if len(pairs) < 2:
             raise ValueError("Not enough Fe atoms for Mode B displacement.")

        def stretch_pair(C, i_fe, i_s, d):
            v = C[i_s] - C[i_fe]
            L = np.linalg.norm(v)
            if L < 1e-6: return C
            u = v/L
            C[i_fe] -= 0.5 * d * u
            C[i_s]  += 0.5 * d * u
            return C

        stretch_pair(new_positions, pairs[0][0], pairs[0][1], +DISPLACEMENT)
        stretch_pair(new_positions, pairs[1][0], pairs[1][1], -DISPLACEMENT)

        return new_positions

    @staticmethod
    def atom_list_to_pyscf_string(symbols, coords):
        return "; ".join([f"{s} {c[0]:.5f} {c[1]:.5f} {c[2]:.5f}"
                          for s, c in zip(symbols, coords)])

# ------------------------------------------
# 3. FUNCTION: ROBUST ANALYSIS (GPU-ENABLED)
# ------------------------------------------
def analyze_and_save(symbols, coords, name, file_tag):
    print(f" [Calc] Running DFT ({BASIS_SET}) for {name}...")

    # 1. Build Molecule
    atom_string = FeS_Analyzer.atom_list_to_pyscf_string(symbols, coords)
    mol = gto.M(atom=atom_string, basis=BASIS_SET, charge=0, spin=0, verbose=0, unit='Angstrom')
    mol.build()

    # 2. Initialize and Move to GPU 🚀
    mf = dft.RKS(mol)
    mf.xc = FUNCTIONAL
    mf.max_cycle = DIIS_MAX_CYCLE
    mf.conv_tol = CONV_TOL

    # *** GPU ACCELERATION LINE ***
    try:
        mf = mf.to_gpu()
        print(" [INFO] GPU acceleration enabled.")
    except Exception as e:
        print(f" [WARNING] GPU acceleration failed ({e}). Reverting to CPU.")

    mf.kernel()

    # 3. Fallback: Newton-Raphson if DIIS failed
    if not mf.converged:
        print(f" [!] DIIS failed. Switching to Newton-Raphson solver via mf.newton()...")

        # If running on GPU, convert back to CPU before using NR, then convert back.
        try:
            mf = mf.to_cpu()
        except:
            pass # Already on CPU or failed conversion

        mf = mf.newton()
        mf.max_cycle = 100
        mf.kernel()

        if not mf.converged:
            print(f" [CRITICAL FAILURE] SCF did not converge for {name} with NR. Skipping...")
            return None, None
        else:
            print(f" [Success] SCF Converged with NR (E={mf.e_tot:.4f} Ha)")
    else:
        print(f" [Success] SCF Converged (E={mf.e_tot:.4f} Ha)")

    # --- Calculation converged, now perform post-processing (on CPU) ---

    # Ensure mf object is on CPU for Numint and NumPy post-processing
    try:
        if hasattr(mf, 'to_cpu'): mf = mf.to_cpu()
    except:
        pass

    # 4. Grid & Laplacian Calculation (32k points)
    coords_np = mol.atom_coords() # Use NumPy coords
    min_c = coords_np.min(axis=0) - BOX_MARGIN
    max_c = coords_np.max(axis=0) + BOX_MARGIN
    xs = np.linspace(min_c[0], max_c[0], GRID_POINTS)
    mx, my, mz = np.meshgrid(xs, xs, xs, indexing='ij')
    grid_coords = np.vstack([mx.ravel(), my.ravel(), mz.ravel()]).T

    # Calculate Density & Laplacian
    ao = mol.eval_gto("GTOval_sph", grid_coords)
    rho = dft.numint.eval_rho(mol, ao, mf.make_rdm1(), xctype=FUNCTIONAL)
    rho_cube = rho.reshape(GRID_POINTS, GRID_POINTS, GRID_POINTS)

    dx = xs[1] - xs[0]
    lap = (np.gradient(np.gradient(rho_cube, axis=0), axis=0)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=1), axis=1)/dx**2 +
           np.gradient(np.gradient(rho_cube, axis=2), axis=2)/dx**2)
    abs_lap = np.abs(lap.ravel())

    # 5. Filter & Save Raw Laplacian Data (kappa)
    cap = np.percentile(abs_lap, 99.5)
    kappa = abs_lap[abs_lap < cap]

    outfile = f"{file_tag}_{BASIS_SET}.npz"
    np.savez_compressed(outfile,
                        kappa=kappa,
                        symbols=symbols,
                        coords=coords)
    print(f" [Saved] -> {outfile} (Raw Laplacian data saved)")

    return kappa, mf

# ------------------------------------------
# 4. ORCHESTRATOR & EXECUTION (Unchanged)
# ------------------------------------------
def fetch_pdb_if_missing(pdb_code):
    filename = f"{pdb_code.lower()}.cif"
    if os.path.exists(filename):
        return filename

    url = f"https://files.rcsb.org/download/{pdb_code}.cif"
    urllib.request.urlretrieve(url, filename)
    return filename

def generate_delta_teeth_json(kappa_base, kappa_mode, tag):
    rows = []
    for b in BINS_LIST:
        min_k = min(kappa_base.min(), kappa_mode.min())
        max_k = max(kappa_base.max(), kappa_mode.max())
        edges = np.linspace(min_k, max_k, b + 1)

        hist_b, _ = np.histogram(kappa_base, bins=edges, density=True)
        hist_m, _ = np.histogram(kappa_mode, bins=edges, density=True)

        tb = int(np.sum(hist_b < 1e-9))
        tm = int(np.sum(hist_m < 1e-9))

        Delta_teeth = tm - tb
        Delta_r_A = L_EFF / b

        rows.append({
            "bins": b,
            "Delta_r_A": Delta_r_A,
            "Delta_teeth": Delta_teeth
        })

    json_name = f"{tag}_DeltaTeeth.json"
    import json
    with open(json_name, "w") as f:
        json.dump(rows, f, indent=2)
    print(f" [Saved] {json_name}")


def run_pipeline(active_path, inhib_path):
    print(f"\n{'='*40}\n=== STARTING CURVATURE RUN ({BASIS_SET}) ===\n{'='*40}")

    act_ana = FeS_Analyzer(active_path)
    act_syms, act_coords = act_ana.extract_cluster()
    inh_ana = FeS_Analyzer(inhib_path)
    inh_syms, inh_coords = inh_ana.extract_cluster()

    act_dist_coords = act_ana.apply_mode_b_distortion(act_syms, act_coords)
    inh_dist_coords = inh_ana.apply_mode_b_distortion(inh_syms, inh_coords)

    print("\n--- Processing Active Cluster (4S39) ---")
    k_act_base, mf1 = analyze_and_save(act_syms, act_coords, "Active Base", "4S39_active_base")
    k_act_dist, mf2 = analyze_and_save(act_syms, act_dist_coords, "Active Distorted", "4S39_active_dist")

    print("\n--- Processing Inhibited Cluster (4S3E) ---")
    k_inh_base, mf3 = analyze_and_save(inh_syms, inh_coords, "Inhibited Base", "4S3E_inhib_base")
    k_inh_dist, mf4 = analyze_and_save(inh_syms, inh_dist_coords, "Inhibited Distorted", "4S3E_inhib_dist")

    print("\n--- Generating Final Data Curves ---")
    if all(k is not None for k in [k_act_base, k_act_dist, k_inh_base, k_inh_dist]):
        generate_delta_teeth_json(k_act_base, k_act_dist, "4S39_Active_Curve")
        generate_delta_teeth_json(k_inh_base, k_inh_dist, "4S3E_Inhibited_Curve")

        def report_teeth(kappa, bins=200):
            hist, _ = np.histogram(kappa, bins=bins, density=True)
            return int(np.sum(hist < 1e-9))

        t_act_base = report_teeth(k_act_base)
        t_act_dist = report_teeth(k_act_dist)
        t_inh_base = report_teeth(k_inh_base)
        t_inh_dist = report_teeth(k_inh_dist)

        delta_act = t_act_dist - t_act_base
        delta_inh = t_inh_dist - t_inh_base

        print(f"\n{'='*40}")
        print("SUMMARY (Delta Teeth at ~0.015 A Resolution)")
        print(f"Active (4S39) Delta: {delta_act}")
        print(f"Inhibited (4S3E) Delta: {delta_inh}")
        print(f"{'='*40}\n[Expected: Delta Active > 0, Delta Inhibited < 0 (Collapse Confirmed)]")
    else:
        print("\n*** ERROR: Failed to generate final curves due to critical SCF failure. ***")


if __name__ == "__main__":
    if 'google.colab' in sys.modules:
        # Install GPU-enabled PySCF (Requires GPU runtime)
        !pip install -q gemmi pyscf
        # It's essential to install cupy and other GPU dependencies for PySCF to work fully.
        # This part assumes a standard Colab GPU runtime which has CUDA pre-installed.
        !pip install cupy-cuda11x # Install compatible CuPy for PySCF GPU acceleration

    try:
        p_active = fetch_pdb_if_missing("4S39")
        p_inhib = fetch_pdb_if_missing("4S3E")

        run_pipeline(os.path.abspath(p_active), os.path.abspath(p_inhib))
    except Exception as e:
        print(f"\nCRITICAL FAILURE DURING SETUP: {e}")

#The video [Specifying GEOMETRY of MOLECULE in PySCF [TUTORIAL #3]](https://www.youtube.com/watch?v=1Eyz-iPykDs) demonstrates how to correctly input molecular geometry strings into PySCF, which is a foundational step in your pipeline.
#http://googleusercontent.com/youtube_content/0 *YouTube video views will be stored in your YouTube History, and your data will be stored and used by YouTube according to its [Terms of Service](https://www.youtube.com/static?template=terms)*




Working directory: /content/gqr_fe4s4_ispG
  Using cached cupy_cuda11x-13.6.0-cp312-cp312-manylinux2014_x86_64.whl.metadata (2.4 kB)
Using cached cupy_cuda11x-13.6.0-cp312-cp312-manylinux2014_x86_64.whl (99.7 MB)

=== STARTING CURVATURE RUN (def2-svp) ===
 [Loader] Reading /content/gqr_fe4s4_ispG/4s39.cif...
 Extracted 8 atoms and centered geometry.
 [Loader] Reading /content/gqr_fe4s4_ispG/4s3e.cif...
 Extracted 8 atoms and centered geometry.

--- Processing Active Cluster (4S39) ---
 [Calc] Running DFT (def2-svp) for Active Base...


/usr/local/lib/python3.12/dist-packages/cupy/_environment.py:596: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
/usr/local/lib/python3.12/dist-packages/gpu4pyscf/lib/cutensor.py:158: UserWarning: using cupy as the tensor contraction eng

Library gpu4pyscf not found. You can install this package via
    pip install gpu4pyscf-cuda11x
See more installation info at https://github.com/pyscf/gpu4pyscf
 [WARNING] GPU acceleration failed (libcublas.so.11: cannot open shared object file: No such file or directory). Reverting to CPU.
 [!] DIIS failed. Switching to Newton-Raphson solver via mf.newton()...
 [Success] SCF Converged with NR (E=-6645.0998 Ha)

CRITICAL FAILURE DURING SETUP: not enough values to unpack (expected 2, got 1)


In [ ]:
# This uses the repository URL which sometimes bypasses build issues
!pip install -q git+https://github.com/pyscf/gpu4pyscf.git


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 14.2 MB/s eta 0:00:00
